In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [ ]:
%autosave 0
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook

# Approximate Inference



For many interesting models the evidence  

$$
p(\mathcal{D}|\mathcal{M}_i) = \int p(\mathcal{D}|\mathcal{M}_i, \theta) p(\theta| \mathcal{M}_i) d\theta
$$

and hence the posterior are intractable:

- The integral has no closed-form
- The dimensionality is so big that numerical integration is not feasible

We resort to stochastic or deterministic approximations

- MCMC is computationally demanding but can be exact
- VI scales better but is not exact

# Laplace Approximation


Propose a function of $\theta \in \mathbb{R}^K$

$$
g(\theta) = \log p(\mathcal{D}| \theta) p(\theta)
$$

Do a second order Taylor expansion around $\theta= \hat \theta_{\text{map}}$

$$
\begin{align}
g(\theta) \approx g(\hat \theta_{\text{map}}) &+ (\theta - \hat \theta_{\text{map}})^T \frac{dg}{d\theta}\bigg \rvert_{\theta=\hat \theta_{\text{map}}} \nonumber \\
&+ \frac{1}{2} (\theta - \hat \theta_{\text{map}})^T \frac{d^2 g}{d\theta^2} \bigg \rvert_{\theta=\hat \theta_{\text{map}}} (\theta - \hat \theta_{\text{map}})
\end{align}
$$

- By definition the first derivative evaluated at $\hat \theta_{\text{map}}$ is zero 
- We call the negative Hessian evaluated at $\hat \theta_{\text{map}}$: $\Sigma^{-1} = -\frac{d^2 g}{d\theta^2} (\hat \theta_{\text{map}})$ 

If we plug the approximation in the evidence we can solve the integral

$$
p(\mathcal{D}) \approx  e^{g(\hat \theta_{\text{map}})} \int e^{-  \frac{1}{2} (\theta - \hat \theta_{\text{map}})^T \Sigma^{-1} (\theta - \hat \theta_{\text{map}})} d\theta = e^{g(\hat \theta_{\text{map}})} (2\pi)^{K/2} |\Sigma|^{1/2}
$$

And the posterior

$$
\begin{align}
p(\theta| \mathcal{D}) &= \frac{p(\mathcal{D}|\theta) p(\theta) }{p(\mathcal{D})} \nonumber \\
&\approx \frac{1}{(2\pi)^{K/2} |\Sigma|^{1/2}} e^{-  \frac{1}{2} (\theta - \hat \theta_{\text{map}})^T \Sigma^{-1} (\theta - \hat \theta_{\text{map}})} 
\end{align}
$$

> Laplace method approximates the posterior by a **Multivariate Gaussian** centered in the MAP

Two steps
1. Find the mode (MAP)
1. Evaluate the Hessian at the mode

Note that
- We didn't assume any distribution for the prior or likelihood
- We require that $g$ is continuous and differentiable on $\theta$ 
- We also require that the negative Hessian of $g$ on the MAP is a proper covariance

#### Evidence decomposition 

Using Laplace approximation the log evidence can be decomposed as

$$
\begin{align}
\log p(\mathcal{D}|\mathcal{M}_i) &\approx g(\hat \theta_{\text{map}}) + \log (2\pi)^{K/2} |\Sigma|^{1/2} \nonumber \\
&=\log p(\mathcal{D}|\mathcal{M}_i, \hat \theta_{\text{map}}) + \log p(\hat \theta_{\text{map}}| \mathcal{M}_i) + \frac{K}{2} \log(2\pi) + \frac{1}{2} \log | \Sigma | \nonumber 
\end{align}
$$

> The log evidence is approximated by the best likelihood fit plus the Occam's factor

The Occam's factor depends on the
- log pdf of $\theta$
- number of parameters $K$
- second derivative of the posterior (model uncertainty)


> If the prior is very broad and $N$ is very large we recover the **Bayesian Information Criterion** (BIC) (Proof?)

# Variational Inference (VI)

We want the posterior

$$
p(\theta|\mathcal{D}) = \frac{p(\mathcal{D}|\theta) p(\theta)}{p(\mathcal{D})}
$$

but it may be intractable

In VI a simpler (tractable) posterior distribution is proposed

$$
q_\eta(\theta)
$$

> We approximate $p(\theta|\mathcal{D})$ with $q_\eta(\theta)$

$q_\eta(\theta)$ represents a family of distributions parametrized by $\eta$

> **Optimization problem:** Find $\eta$ that makes $q$ most similar to $p$

We can write this as a KL divergence

$$
\min_\eta D_{\text{KL}}[q_\eta(\theta) || p(\theta|\mathcal{D})] = \int q_\eta(\theta) \log \frac{q_\eta(\theta)}{p(\theta|\mathcal{D})} d\theta
$$

This is still intractable!

To continue we use Bayes Theorem on the posterior and move the evidence out from the integral

$$
D_{\text{KL}}[q_\eta(\theta) || p(\theta|\mathcal{D})] = \log p(\mathcal{D}) + \int q_\eta(\theta) \log \frac{q_\eta(\theta)}{p(\mathcal{D}|\theta) p (\theta)} d\theta
$$

> If we are minimizing with respect to $\eta$ we can ignore the evidence

The KL divergence is 
- non-negative
- zero only if $q_\eta(\theta) \equiv p(\theta|\mathcal{D})$

Using the non-negativity we find a lower bound for the evidence

$$
\log p(\mathcal{D}) \geq  \mathcal{L}(\eta) = - \int q_\eta(\theta) \log \frac{q_\eta(\theta)}{p(\mathcal{D}|\theta) p (\theta)} d\theta
$$

> $\mathcal{L}(\eta)$ is called the **Evidence Lower BOund** (ELBO)

- Minimizing the KL between $q$ and $p$ is equivalent to maximizing the ELBO wrt $q$
$$
\hat \eta = \text{arg}\max_\eta \mathcal{L}(\eta)
$$
- We can use $q_{\hat \eta}(\theta)$ as a drop-in replacement for $p(\theta|\mathcal{D})$
- The ELBO is tractable for simple, parametric $q$
- The ELBO can only be tight if $p$ is within the family of $q$



¿Why variational?

- Functional: Function of functions. [Calculus of variations](https://en.wikipedia.org/wiki/Calculus_of_variations): Derivatives of functionals
- [Variational Free Energy](https://en.wikipedia.org/wiki/Thermodynamic_free_energy): $-\mathcal{L}(\eta)$ 


### Another way to "obtain" the ELBO

Using Jensen's inequality on the log evidence

$$
\begin{align}
\log p(\mathcal{D}) &=  \log \mathbb{E}_{\theta\sim p(\theta)} \left[p(\mathcal{D}|\theta)\right]\nonumber \\
&=  \log \mathbb{E}_{\theta\sim q_\eta(\theta)} \left[p(\mathcal{D}|\theta)\frac{p(\theta)}{q_\eta(\theta)}\right]\nonumber \\
&\geq  \mathbb{E}_{\theta\sim q_\eta(\theta)} \left[\log p(\mathcal{D}|\theta)\frac{p(\theta)}{q_\eta(\theta)}\right] =- \int q_\eta(\theta) \log \frac{q_\eta(\theta)}{p(\mathcal{D}|\theta) p (\theta)} d\theta \nonumber 
\end{align}
$$

### Fully-factorized posterior

A simple (tractable) posterior

$$
q_\eta(\theta) = \prod_{i=1}^K q_{\eta_i}(\theta_i)
$$

- no correlation between factors
- this is known as the Mean-field VI or Mean-field Theory (physics)

Using this factorized posterior the ELBO
$$
\mathcal{L}(\eta) =  \int q_{\eta_i}(\theta_i) \log \left [ p(\mathcal{D}|\theta)p(\theta) \prod_{j\neq i} q_{\eta_j}(\theta_j) d\theta_j \right ] d\theta_i - \sum_i \int q_{\eta_i}(\theta_i) \log q_{\eta_i}(\theta_i)  d\theta_i
$$

We can  keep all $\theta$ but $i$ fixed and update $i$ iteratively

$$
\begin{align}
\mathcal{L}(\eta_i) &=  \int q_{\eta_i}(\theta_i) \mathbb{E}_{\prod q_{i\neq j}} \left[ p(\mathcal{D}| \theta)p(\theta) \right ] d\theta_i - \int q_{\eta_i}(\theta_i) \log q_{\eta_i}(\theta_i)  d\theta_i + \text{Constant}  \nonumber \\
&  - \int q_{\eta_i}(\theta_i) \log \frac{ q_{\eta_i}(\theta_i)}{\mathbb{E}_{\prod q_{i\neq j}} \left[ p(\mathcal{D}| \theta)p(\theta) \right ]} d\theta_i + \text{Constant} \nonumber
\end{align}
$$

> Maximizing the ELBO es equivalent to minimizing the KL between $q_{\eta_i}(\theta_i)$ and 

The solution is

$$
\log q_i(\eta_i) =  \mathbb{E}_{\prod q_{i\neq j}} \left[ p(\mathcal{D}| \theta)p(\theta) \right ] + \text{Constant}
$$
> Guaranteed convergence (convex on factors $q_i$)

## Example: Approximating a Gaussian

A full univariate Gaussian model would be

$$
\begin{align}
p(x, \theta) &= p(x|\theta) p(\theta) \nonumber \\
& = p(x|\mu, \sigma^2) p(\mu, \sigma^2) \nonumber \\
& = p(x|\mu, \sigma^2) p(\mu|\sigma^2) p(\sigma^2) \nonumber \\
\end{align}
$$ 

A full conjugate model would have Normal $p(\mu|\sigma^2) $ and Gamma $p(\sigma^2)$

Approximating with a fully-factorized assumes

$$
q(\theta) = q(\mu)q(\sigma^2)
$$

In [ ]:
import pyro
import torch
from pyro.distributions import Normal, InverseGamma
from torch.distributions import constraints
from pyro.infer import EmpiricalMarginal


from pyro.infer.mcmc.api import MCMC
from pyro.infer.mcmc import NUTS

data = 5. + 2.*torch.from_numpy(np.random.randn(10000).astype('float32'))

def model(x):
    s2 = pyro.sample("s2", InverseGamma(torch.tensor(0.1), torch.tensor(0.1)))    
    mu = pyro.sample("mu", Normal(torch.tensor(0.), s2))  
    return pyro.sample("likelihood", Normal(mu, s2), obs=x)

nuts_kernel = NUTS(model, adapt_step_size=True)
sampler = MCMC(nuts_kernel, num_chains=2, num_samples=1000, warmup_steps=100)
sampler.run(data)

In [ ]:
sampler.diagnostics()
samples = sampler.get_samples(10000)
mu_plot = samples['mu'].detach().numpy()
s2_plot = samples['s2'].detach().numpy()

In [ ]:
import corner
#ax.hist2d(samples['mu'].detach().numpy(), samples['s2'].detach().numpy(), 
#          bins=20, cmap=plt.cm.Blues, range=[[4.5, 5.5], [1.5, 2.5]])
#ax.set_xlabel('mu')
#ax.set_ylabel('s2')
figure = corner.corner(np.stack((mu_plot, s2_plot)).T, 
                       labels=[r"$\mu$", r"$\sigma^2$"],
                       quantiles=[0.16, 0.5, 0.84],
                       show_titles=True, title_kwargs={"fontsize": 12})

In [ ]:
def guide(x):
    mu_loc = pyro.param("mu_loc", torch.tensor(0.))
    mu_scale = pyro.param("mu_scale", torch.tensor(1.), constraint=constraints.positive)
    mu = pyro.sample("mu", Normal(mu_loc, mu_scale))  
    a = pyro.param("s_a", torch.tensor(1.), constraint=constraints.positive)
    b = pyro.param("s_b", torch.tensor(1.), constraint=constraints.positive)
    s2 = pyro.sample("s2", InverseGamma(a, b))    


pyro.clear_param_store()
svi = pyro.infer.SVI(model=model, guide=guide,
                     optim=pyro.optim.Adam({"lr": 0.05}),
                     loss=pyro.infer.Trace_ELBO(), num_samples=100)

In [ ]:

fig, ax = plt.subplots(2, 3, figsize=(7, 3), tight_layout=True)
lines = [ax_.plot([], [])[0] for ax_ in ax.ravel()[:-1]]
param_names = ["mu_loc", "mu_scale", "s_a", "s_b"]
losses, param_evolution = [], {}
for name in param_names:
    param_evolution[name] = []
    
ax[0, 0].set_title('ELBO')
for ax_, name in zip(ax.ravel()[1:], param_names):
    ax_.set_title(name)

for k in tqdm_notebook(range(7500)):
    losses.append(svi.step(data))
    #display(pyro.get_param_store().keys())

    for name in param_names:
        param_evolution[name].append(pyro.param(name).item())
    
    

    if np.mod(k, 100) == 0:
        lines[0].set_ydata(losses[:k])
        for i, name in enumerate(param_names):
            lines[i+1].set_ydata(param_evolution[name][:k])
        for line in lines:
            line.set_xdata(range(k))
        for ax_ in ax.ravel():
            ax_.relim()
            ax_.autoscale_view()
        ax[1, 2].cla() 
        posterior = svi.run(data)
        mu_plot = EmpiricalMarginal(posterior, "mu").sample(sample_shape=torch.Size([10000]))
        s2_plot = EmpiricalMarginal(posterior, "s2").sample(sample_shape=torch.Size([10000]))
        ax[1, 2].hist2d(mu_plot.detach().numpy(), s2_plot.detach().numpy(), 
                        bins=20, cmap=plt.cm.Blues, range=[[3.5, 6.5], [0., 5.]])
        ax[1, 2].set_ylim([0, 5.])
        ax[1, 2].set_xlim([3.5, 6.5])
        fig.canvas.draw()

In [ ]:
from pyro.infer import EmpiricalMarginal
posterior = svi.run(data)
mu_plot = EmpiricalMarginal(posterior, "mu").sample(sample_shape=torch.Size([10000]))
s2_plot = EmpiricalMarginal(posterior, "s2").sample(sample_shape=torch.Size([10000]))

fig, ax = plt.subplots()
ax.hist2d(mu_plot.detach().numpy(), 
          s2_plot.detach().numpy(), bins=20, cmap=plt.cm.Blues)
ax.set_xlabel('mu')
ax.set_ylabel('s2')

## More attention on the ELBO 

$$
\begin{align}
\mathcal{L}(\eta) &= - \int q_\eta(\theta) \log \frac{q_\eta(\theta)}{p(\mathcal{D}|\theta) p (\theta)} d\theta \nonumber \\
&= - \int q_\eta(\theta) \log \frac{q_\eta(\theta)}{ p (\theta)} d\theta + \int q_\eta(\theta) \log p(\mathcal{D}|\theta) d\theta \nonumber \\
&= - D_{KL}[q_\eta(\theta) || p(\theta)] + \mathbb{E}_{\theta \sim q_\eta(\theta)} \left[\log p(\mathcal{D}|\theta)\right]\nonumber 
\end{align}
$$

> Maximizing the ELBO is equivalent to:
- Maximize the log likelihood when sampling from the approximate posterior
- Minimize the KL between the approximate posterior and prior

# Self-study

- [Chapter 28 of D. Barber's book](http://web4.cs.ucl.ac.uk/staff/D.Barber/pmwiki/pmwiki.php?n=Brml.Online)
    - 28.2 Laplaca Approximation
    - 28.4 Variational Bounding
- David Blei, ["Variational Inference: A review for statisticians"](https://arxiv.org/abs/1601.00670), ["Foundations and innovations"](https://www.youtube.com/watch?v=DaqNNLidswA)
- https://cedar.buffalo.edu/~srihari/CSE574/Chap10/10.2VariationalInference.pdf